In [2]:
import pandas as pd
data = pd.read_csv("cleaned_file.csv")

In [4]:
nan_count = data.isna().sum().sum()
nan_count

0

In [ ]:
zero_count = (data == 0).sum().sum()
zero_count
# this is going to be a lot of invalid

1189693

In [9]:
mortality_col = [col for col in data.columns if 'cvd' in col.lower()]
mortality_col

['Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18',
 'Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20',
 'Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21',
 'Female_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18',
 'Female_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20',
 'Female_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21',
 'Male_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18',
 'Male_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20',
 'Male_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21',
 'Black_AA_Single_Race_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20',
 'Asian_Single_Race_Female_CVD_Deaths_Counts_CDC2018_22_x',
 'Asian_Single_Race_Male_CVD_Deaths_Counts_CDC2018_22_x',
 'NHPI_Single_Race_Female_CVD_Deaths_Counts_CDC2018_22_x',
 'NHPI_Single_Race_Male_CVD_Deaths_Counts_CDC2018_22_x']

In [30]:
#combine all three cvds
data["Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_2021"] = (
    data["Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18"] +
    data['Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20'] +
    data['Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21']
) / 3

old_cvd_colums = ["Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_18", 'Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2018_20', 
                  'Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2019_21']
data = data.drop(columns= old_cvd_colums)


In [51]:
data.drop(data.columns[0], axis = 1)
data

,AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,Total_population_16yr_And_Over_For_Work_Commute_ACS2017_21,Total_population_With_Daily_Commute_Less_Than_15min_ACS2017_21,...,White_Single_Race_Female_Percent_of_Hypertension_In_Total_Deaths_CDC2018_22_x,"White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22",Number_of_hospitals_County_CDCnehTracking_2020,Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20,stname,Water Area km²,Water Area mi²,Latitude,Longitude,"Total_Population_CVD_Death_Rate_Per_100,000_Residents_CDC_2016_2021"
0,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,6085.0,...,0.0,35.5,1.0,11.9,Alabama,25.776,9.952,32.536382,-86.644490,282.466667
1,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,24258.0,...,0.0,42.6,4.0,15.0,Alabama,1133.190,437.527,30.659218,-87.746067,239.700000
2,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,3326.0,...,0.0,0.0,1.0,15.9,Alabama,50.865,19.639,31.870670,-85.405456,336.766667
3,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,1733.0,...,0.0,49.4,1.0,18.0,Alabama,9.289,3.587,33.015893,-87.127148,324.500000
4,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,4742.0,...,0.0,53.6,1.0,18.6,Alabama,15.157,5.852,33.977448,-86.567246,298.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,10500.0,...,0.0,30.9,2.0,11.9,Wyoming,166.887,64.436,41.660339,-108.875676,231.400000
3139,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,6118.0,...,0.0,0.0,1.0,0.0,Wyoming,572.266,220.953,44.049321,-110.588102,114.366667
3140,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,5841.0,...,0.0,0.0,2.0,0.0,Wyoming,16.342,6.310,41.284726,-110.558947,224.700000
3141,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,2852.0,...,0.0,0.0,1.0,0.0,Wyoming,10.762,4.155,43.878831,-107.669052,212.333333


In [ ]:
#combine all year columns
import pandas as pd
import re

# Define a regex pattern to match years between 2000 and 2024
year_pattern = r"20(?:0[0-9]|1[0-9]|2[0-4])"  # Matches 2000 to 2024

# Find all columns containing years in their names
columns_with_years = [col for col in data.columns if re.search(year_pattern, col)]

columns_with_years




In [61]:
import pandas as pd
import re

def combine_columns_by_years(df, year_range=(2000, 2024)):
    year_pattern = r"20(?:0[0-9]|1[0-9]|2[0-4])"  # Match years
    grouped_columns = {}

    # Step 1: Normalize column names
    df.columns = (
        df.columns.str.strip()
                 .str.lower()
                 .str.replace(r"[/\s]+", "_", regex=True)  # Replace spaces and slashes
    )

    # Step 2: Group columns based on base name and overlapping year ranges
    for col in df.columns:
        years = re.findall(year_pattern, col)
        if years:
            base_name = re.sub(r"_20\d{2}.*", "", col)  # Remove year suffix
            min_year, max_year = min(map(int, years)), max(map(int, years))
            grouped_columns.setdefault((base_name, min_year, max_year), []).append(col)

    # Step 3: Combine grouped columns
    for (base_name, start, end), cols in grouped_columns.items():
        if len(cols) > 1:  # Only combine if multiple columns exist
            new_col = f"{base_name}_{start}_{end}"
            df[new_col] = df[cols].mean(axis=1)
            df.drop(columns=cols, inplace=True)

    return df
data_less_col = combine_columns_by_years(data)
data_less_col 


,airpollutant:_acetaldehyde_ug_m^3_epa2017,airpollutant:_acetaldehyde_ug_m^3_epa2018,airpollutant:_benzene_ug_m^3_epa2017,airpollutant:_benzene_ug_m^3_epa2018,airpollutant:_carbon_tetrachloride_ug_m^3_epa2017,airpollutant:_carbon_tetrachloride_ug_m^3_epa2018,airpollutant:_formaldehyde_ug_m^3_epa2017,airpollutant:_formaldehyde_ug_m^3_epa2018,total_population_16yr_and_over_for_work_commute_acs2017_21,total_population_with_daily_commute_less_than_15min_acs2017_21,...,number_of_pharmacies_available_per100000_cdcnehtracking_2016_20,stname,water_area_km²,water_area_mi²,latitude,longitude,"total_population_cvd_death_rate_per_100,000_residents_cdc_2016_2021",net_migration_rate_2010_2010,net_migrants_2010_2010,expected_population_2020_2020
0,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,6085.0,...,11.9,Alabama,25.776,9.952,32.536382,-86.644490,282.466667,6.705625,169.3125,3373.380000
1,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,24258.0,...,15.0,Alabama,1133.190,437.527,30.659218,-87.746067,239.700000,27.418750,2949.8125,10897.011875
2,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,3326.0,...,15.9,Alabama,50.865,19.639,31.870670,-85.405456,336.766667,-7.526875,-138.0625,1645.970625
3,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,1733.0,...,18.0,Alabama,9.289,3.587,33.015893,-87.127148,324.500000,-2.642500,-46.5000,1384.510000
4,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,4742.0,...,18.6,Alabama,15.157,5.852,33.977448,-86.567246,298.600000,3.271875,95.8750,3448.338750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,10500.0,...,11.9,Wyoming,166.887,64.436,41.660339,-108.875676,231.400000,-9.998125,-286.0000,2863.797500
3139,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,6118.0,...,0.0,Wyoming,572.266,220.953,44.049321,-110.588102,114.366667,4.732500,33.1875,1382.926875
3140,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,5841.0,...,0.0,Wyoming,16.342,6.310,41.284726,-110.558947,224.700000,-8.278750,-130.4375,1376.253750
3141,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,2852.0,...,0.0,Wyoming,10.762,4.155,43.878831,-107.669052,212.333333,-8.796875,-50.1875,501.933750


In [82]:
data_filtered = data_less_col.loc[:, ~data_less_col.columns.str.contains(r'single_race|multi_race', case=False)]
data_filtered

,airpollutant:_acetaldehyde_ug_m^3_epa2017,airpollutant:_acetaldehyde_ug_m^3_epa2018,airpollutant:_benzene_ug_m^3_epa2017,airpollutant:_benzene_ug_m^3_epa2018,airpollutant:_carbon_tetrachloride_ug_m^3_epa2017,airpollutant:_carbon_tetrachloride_ug_m^3_epa2018,airpollutant:_formaldehyde_ug_m^3_epa2017,airpollutant:_formaldehyde_ug_m^3_epa2018,total_population_16yr_and_over_for_work_commute_acs2017_21,total_population_with_daily_commute_less_than_15min_acs2017_21,...,number_of_pharmacies_available_per100000_cdcnehtracking_2016_20,stname,water_area_km²,water_area_mi²,latitude,longitude,"total_population_cvd_death_rate_per_100,000_residents_cdc_2016_2021",net_migration_rate_2010_2010,net_migrants_2010_2010,expected_population_2020_2020
0,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,6085.0,...,11.9,Alabama,25.776,9.952,32.536382,-86.644490,282.466667,6.705625,169.3125,3373.380000
1,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,24258.0,...,15.0,Alabama,1133.190,437.527,30.659218,-87.746067,239.700000,27.418750,2949.8125,10897.011875
2,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,3326.0,...,15.9,Alabama,50.865,19.639,31.870670,-85.405456,336.766667,-7.526875,-138.0625,1645.970625
3,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,1733.0,...,18.0,Alabama,9.289,3.587,33.015893,-87.127148,324.500000,-2.642500,-46.5000,1384.510000
4,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,4742.0,...,18.6,Alabama,15.157,5.852,33.977448,-86.567246,298.600000,3.271875,95.8750,3448.338750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,10500.0,...,11.9,Wyoming,166.887,64.436,41.660339,-108.875676,231.400000,-9.998125,-286.0000,2863.797500
3139,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,6118.0,...,0.0,Wyoming,572.266,220.953,44.049321,-110.588102,114.366667,4.732500,33.1875,1382.926875
3140,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,5841.0,...,0.0,Wyoming,16.342,6.310,41.284726,-110.558947,224.700000,-8.278750,-130.4375,1376.253750
3141,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,2852.0,...,0.0,Wyoming,10.762,4.155,43.878831,-107.669052,212.333333,-8.796875,-50.1875,501.933750


In [71]:
response_keywords = ['death','cvd']
response_columns = [col for col in data_filtered.columns if any(keyword.lower() in col.lower() for keyword in response_keywords)]
response_columns

['female_population_cvd_death_rate_per_100,000_residents_cdc_2016_18',
 'female_population_cvd_death_rate_per_100,000_residents_cdc_2018_20',
 'female_population_cvd_death_rate_per_100,000_residents_cdc_2019_21',
 'male_population_cvd_death_rate_per_100,000_residents_cdc_2016_18',
 'male_population_cvd_death_rate_per_100,000_residents_cdc_2018_20',
 'male_population_cvd_death_rate_per_100,000_residents_cdc_2019_21',
 'total_population_cvd_death_rate_per_100,000_residents_cdc_2016_2021']

In [ ]:
data_x_less = data_filtered.drop(columns=response_columns)
cols_to_drop = ['water_area_km²', 'water_area_mi²', 'latitude', 'longitude',
                'total_population_cvd_death_rate_per_100,000_residents_cdc_2016_2021',
                'net_migration_rate_2010_2010', 'net_migrants_2010_2010', 'expected_population_2020_2020', 'stname',
                'fips',	'unnamed:_0', 'sort_[1]',	'population_(2010)',	'land_area_km²',	'statecounty_x']

data_x_less = data_x_less.drop(
    columns=[col for col in data_x_less.columns if col.lower() in [c.lower() for c in cols_to_drop]],
    errors='ignore'  
)
data_x_less

,airpollutant:_acetaldehyde_ug_m^3_epa2017,airpollutant:_acetaldehyde_ug_m^3_epa2018,airpollutant:_benzene_ug_m^3_epa2017,airpollutant:_benzene_ug_m^3_epa2018,airpollutant:_carbon_tetrachloride_ug_m^3_epa2017,airpollutant:_carbon_tetrachloride_ug_m^3_epa2018,airpollutant:_formaldehyde_ug_m^3_epa2017,airpollutant:_formaldehyde_ug_m^3_epa2018,total_population_16yr_and_over_for_work_commute_acs2017_21,total_population_with_daily_commute_less_than_15min_acs2017_21,...,percent_of_farmers_market_accept_snap_usda_2018,number_of_farmers_market_accept_wic_usda_2018,percent_of_farmers_market_accept_wic_usda_2018,number_of_farmers_market_accept_wic_cash_usda_2018,number_of_farmers_market_accept_sfmnp_usda_2018,percent_of_farmers_market_accept_sfmnp_usda_2018,number_of_recreation_fitness_facilities_per1000_residents_usda_2016,populations_vulnerable_to_predicted_surface_smoke_from_wildland_fires_noaa2020,number_of_hospitals_county_cdcnehtracking_2020,number_of_pharmacies_available_per100000_cdcnehtracking_2016_20
0,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,6085.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,6.0,59759.0,1.0,11.9
1,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,24258.0,...,0.000000,0.0,0.0,0.0,4.0,100.0,21.0,246435.0,4.0,15.0
2,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,3326.0,...,50.000000,0.0,0.0,0.0,2.0,50.0,0.0,24706.0,1.0,15.9
3,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,1733.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,22005.0,1.0,18.0
4,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,4742.0,...,0.000000,1.0,100.0,0.0,1.0,100.0,4.0,59512.0,1.0,18.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,10500.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,6.0,41345.0,2.0,11.9
3139,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,6118.0,...,33.333332,0.0,0.0,0.0,0.0,0.0,13.0,23287.0,1.0,0.0
3140,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,5841.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,2.0,20712.0,2.0,0.0
3141,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,2852.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,7719.0,1.0,0.0


In [ ]:
columns_to_combine = {
    'airpollutant:_acetaldehyde_ug_m^3_epa2017': 'airpollutant:_acetaldehyde_ug_m^3_epa2018',
    'airpollutant:_benzene_ug_m^3_epa2017': 'airpollutant:_benzene_ug_m^3_epa2018',
    'airpollutant:_carbon_tetrachloride_ug_m^3_epa2017': 'airpollutant:_carbon_tetrachloride_ug_m^3_epa2018',
    'airpollutant:_formaldehyde_ug_m^3_epa2017': 'airpollutant:_formaldehyde_ug_m^3_epa2018',
}

for col_2017, col_2018 in columns_to_combine.items():
    new_col_name = col_2017.replace('2017', '2017_2018')  # Rename combined column
    data_x_less[new_col_name] = (data_x_less[col_2017] + data_x_less[col_2018]) / 2  # Compute average
    data_x_less = data_x_less.drop(columns=[col_2017, col_2018])  # Drop the original columns

In [92]:
data_x_less

,total_population_16yr_and_over_for_work_commute_acs2017_21,total_population_with_daily_commute_less_than_15min_acs2017_21,total_population_with_daily_commute_between_15_60min_acs2017_21,total_population_with_daily_commute_60min_and_above_acs2017_21,total_population_percent_with_daily_commute_less_than_15min_acs2017_21,total_population_percent_with_daily_commute_between_15_60min_acs2017_21,total_population_percent_with_daily_commute_60min_and_above_acs2017_21,total_population_16yr_and_over_commute_to_work_acs2017_21,total_population_drove_alone_to_work_acs2017_21,total_population_carpool_to_work_acs2017_21,...,number_of_farmers_market_accept_sfmnp_usda_2018,percent_of_farmers_market_accept_sfmnp_usda_2018,number_of_recreation_fitness_facilities_per1000_residents_usda_2016,populations_vulnerable_to_predicted_surface_smoke_from_wildland_fires_noaa2020,number_of_hospitals_county_cdcnehtracking_2020,number_of_pharmacies_available_per100000_cdcnehtracking_2016_20,airpollutant:_acetaldehyde_ug_m^3_epa2017_2018,airpollutant:_benzene_ug_m^3_epa2017_2018,airpollutant:_carbon_tetrachloride_ug_m^3_epa2017_2018,airpollutant:_formaldehyde_ug_m^3_epa2017_2018
0,25204.0,6085.0,17407.0,1712.0,24.14,69.06,6.79,26458.0,22860.0,1818.0,...,0.0,0.0,6.0,59759.0,1.0,11.9,1.455,0.185,0.440,1.765
1,94637.0,24258.0,63185.0,7194.0,25.63,66.77,7.60,102650.0,83559.0,8454.0,...,4.0,100.0,21.0,246435.0,4.0,15.0,1.125,0.165,0.440,1.435
2,8324.0,3326.0,4020.0,978.0,39.96,48.29,11.75,8461.0,7033.0,1009.0,...,2.0,50.0,0.0,24706.0,1.0,15.9,1.450,0.175,0.435,1.735
3,7626.0,1733.0,5029.0,864.0,22.72,65.95,11.33,7994.0,6952.0,619.0,...,0.0,0.0,1.0,22005.0,1.0,18.0,1.575,0.180,0.440,1.880
4,23240.0,4742.0,14531.0,3967.0,20.40,62.53,17.07,23918.0,20670.0,2210.0,...,1.0,100.0,4.0,59512.0,1.0,18.6,1.215,0.195,0.435,1.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,20061.0,10500.0,8281.0,1280.0,52.34,41.28,6.38,20579.0,15801.0,2902.0,...,0.0,0.0,6.0,41345.0,2.0,11.9,0.305,0.100,0.365,0.500
3139,12795.0,6118.0,6385.0,292.0,47.82,49.90,2.28,14590.0,9501.0,728.0,...,0.0,0.0,13.0,23287.0,1.0,0.0,0.335,0.070,0.355,0.435
3140,9236.0,5841.0,2556.0,839.0,63.24,27.67,9.08,9606.0,7610.0,1297.0,...,0.0,0.0,2.0,20712.0,2.0,0.0,0.285,0.080,0.355,0.490
3141,3715.0,2852.0,753.0,110.0,76.77,20.27,2.96,3826.0,3102.0,292.0,...,0.0,0.0,1.0,7719.0,1.0,0.0,0.375,0.095,0.390,0.520


In [94]:
data_x_less.to_csv("x_imputed_less_response.csv")

In [95]:
data_y = data_filtered['total_population_cvd_death_rate_per_100,000_residents_cdc_2016_2021']
data_y.to_csv("response.csv")